In [1]:
import requests
from dotenv import load_dotenv
import os 
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
# from MCForecastTools import MCSimulation
import json
import matplotlib.pyplot as plt

load_dotenv()

True

In [2]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version= "v2")


In [3]:
current_date = pd.Timestamp("2020-01-05", tz="America/New_York").isoformat()

tickers = ["SPY", "AAPL", "TSLA", "GE", "F", "MSFT","NIO", "PFE", "AMZN","AAL","DIS","PLUG"]


timeframe = "1D"

In [4]:
def get_data(ticker):
    df = alpaca.get_barset(
        ticker,
        timeframe,
        start = current_date
    ).df

    return df


In [5]:
def get_columns(dataframe):
    dataframe = dataframe.T.reset_index(drop = True).T
    dataframe = dataframe.rename(columns={3:"close", 4:"volume"})
    dataframe = dataframe.loc[:,["close", "volume"]]

    return dataframe

In [6]:
def calc_daily_returns(dataframe):
    dataframe["pct_change"] = dataframe["close"].pct_change()
    dataframe["volatility"] = np.std(dataframe["pct_change"])

    return dataframe


In [7]:
df_spy = get_data("SPY")
df_spy.head()

SPY                                  
                               open    high     low   close    volume
time                                                                 
2020-01-06 00:00:00-05:00  320.4900  323.73  320.36  323.67  43105385
2020-01-07 00:00:00-05:00  323.0200  323.54  322.24  322.76  33909984
2020-01-08 00:00:00-05:00  322.9400  325.78  322.67  324.42  56581814
2020-01-09 00:00:00-05:00  326.1600  326.73  325.52  326.62  43365068
2020-01-10 00:00:00-05:00  327.2899  327.46  325.20  325.70  44989039

In [8]:
df_spy = get_columns(df_spy)

In [9]:
df_spy = calc_daily_returns(df_spy)

In [12]:
df_spy.tail(30)

,close,volume,pct_change,volatility
time,,,,
2020-11-25 00:00:00-05:00,362.6500,35309156.0,-0.001487,0.021466
2020-11-27 00:00:00-05:00,363.6500,46840189.0,0.002757,0.021466
2020-11-30 00:00:00-05:00,362.0000,67154075.0,-0.004537,0.021466
2020-12-01 00:00:00-05:00,365.9700,64527332.0,0.010967,0.021466
2020-12-02 00:00:00-05:00,366.7599,37323404.0,0.002158,0.021466
2020-12-03 00:00:00-05:00,366.6698,52905586.0,-0.000246,0.021466
2020-12-04 00:00:00-05:00,369.8300,40264568.0,0.008619,0.021466
2020-12-07 00:00:00-05:00,369.1200,40456544.0,-0.001920,0.021466
2020-12-08 00:00:00-05:00,370.1599,32702217.0,0.002817,0.021466
